Sources for data are as follows:

Average time child spends playing a sport and quitting: https://www.aspenprojectplay.org/youth-sports/facts/challenges
How often adults aged 16-34 did physical activity: https://sportengland-production-files.s3.eu-west-2.amazonaws.com/s3fs-public/2022-06/Activity%20Check-in%20-%20Wave%201%20findings%20report.pdf?VersionId=pV4.G3_TTnpfCM5rdLlhJL33EI4cgnBN
How far do people usually travel to go to the gym: https://www.wsj.com/articles/how-close-do-you-need-to-be-to-your-gym-1490111186
(4 miles or 6.44 kilometers 2km deviation ~ think about this)


In [80]:
%pip install numpy
%pip install pandas
%pip install lightfm==1.16

[notice] A new release of pip is available: 23.0.1 -> 23.1.2


[notice] To update, run: C:\Users\Ruslan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2

[notice] To update, run: C:\Users\Ruslan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


[notice] A new release of pip is available: 23.0.1 -> 23.1.2

[notice] To update, run: C:\Users\Ruslan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

In [69]:
import numpy as np
from asgiref.sync import sync_to_async
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import recall_at_k
import random
import datetime
import os
import django
from datetime import date
from scipy import sparse

#View more columns with pandas
pd.set_option('display.max_columns', None)

#Setup Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "TeamBackend.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

#Set the number of users and items for synthetic data gen
statSize = 500

In [70]:
participationLength = np.array([3.3, 3.2, 2.5, 5.1, 4.1, 4.1, 2.5, 2.8, 3.0, 3.1, 2.2, 2.6, 2.8, 4.3, 3.0, 2.8, 3.2, 1.9, 1.6])
std = np.std(participationLength)
avg = np.mean(participationLength)

reasonsValues = np.array([25.0, 23, 19, 18, 14, 13, 6])
wholeSum = np.sum(reasonsValues)
for i in range(len(reasonsValues)):
    reasonsValues[i] = reasonsValues[i]/wholeSum

cleanReasons = np.array([reasonsValues[0], reasonsValues[1], reasonsValues[2] + reasonsValues[3], reasonsValues[5], reasonsValues[4] + reasonsValues[6]])

def estimate_normal_distribution(hist_counts, bin_edges):
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    mu = np.average(bin_centers, weights=hist_counts)
    var = np.average((bin_centers - mu)**2, weights=hist_counts)
    sigma = np.sqrt(var)
    return mu, sigma

hist_counts = np.array([10.3, 19.83, 19.72, 6.44, 7.88])
bin_edges = np.array([6, 18, 35, 55, 65, 99])
mu, sigma = estimate_normal_distribution(hist_counts, bin_edges)

sportNames = [x.upper() for x in ['Cricket', 'Football', 'Volleyball', 'Field Hockey', 'Tennis', 'Basketball', 'Table Tennis', 'Baseball', 'Track and Field', 'Golf', 'Rugby', 'Badminton', 'American Football', 'Swimming', 'Gymnastics', 'Cycling', 'Ice Hockey', 'Handball', 'Rock Climbing', 'Frisbee', 'Bowling']]
sportParticipation = [580, 1259.5, 338.1, 162.2, 1228.5, 995.6, 136.5, 57, 0, 1226, 620.3, 226, 0, 3541.8, 2.4, 2941.2, 2.4, 0, 300, 0, 9.6]
for i in range(len(sportParticipation)):
    if (sportParticipation[i] == 0):
        sportParticipation[i] = 5

sportParticipationasNpArray = np.array(sportParticipation)
sportParticipationSum = sum(sportParticipationasNpArray)
sportOdds = sportParticipation

for i in range(len(sportOdds)):
    sportOdds[i] = sportOdds[i]/sportParticipationSum


In [93]:
df = pd.DataFrame()

genderOptions = ['M', 'F', 'O']
genderProbability = [0.495, 0.495, 0.01]

yearsOfExperience = np.random.normal(avg, std, statSize)
yearsOfExperience[yearsOfExperience < 0] = 0

daysTrainedPerWeek = [0, 1, 2, 3, 4, 5]
daysTrainedProbability = [0.08, 0.15, 0.15, 0.19, 0.18, 0.25]

indoorOutdoorChoices = ["Indoor", "Outdoor"]
indoorOutdoorProbability = [0.5, 0.5]

experienceCategory = yearsOfExperience
boundaries = [4, 3, 2, 1, 0]
category_names = ['ADVANCED', 'LOWER INTERMEDIATE', 'UPPER INTERMEDIATE', 'INTERMEDIATE', 'BEGINNER']
assignment = np.select(
    condlist=[experienceCategory >= b for b in boundaries],
    choicelist=category_names,
    default=np.nan
)
assignment = assignment.astype(str)

exerciseReasons = ['Fitness', 'Importance of Exercise', 'Mental Happiness and Stress', 'Challenging Myself', 'Other']
exerciseReasonsProbability = cleanReasons

ageDistribution = np.random.normal(mu, sigma, statSize)
ageDistribution = np.ceil(ageDistribution)

def assign_age_range(age):
    if 18 <= age <= 25:
        return '18-25'
    elif 26 <= age <= 35:
        return '26-35'
    elif 36 <= age <= 45:
        return '36-45'
    elif 46 <= age <= 55:
        return '46-55'
    elif 56 <= age <=65:
        return '56-65'
    else:
        return 'Over 65'

#df['Gender'] = np.random.choice(genderOptions, size = statSize, p=genderProbability)
#df['YearsOfExperience'] = yearsOfExperience
#df['DaysPerWeek'] = np.random.choice(daysTrainedPerWeek, size = statSize, p=daysTrainedProbability)
#df['IndoorOutdoor'] = np.random.choice(indoorOutdoorChoices, size = statSize, p=indoorOutdoorProbability)
#df['ReasonForExercising'] = np.random.choice(exerciseReasons, size = statSize, p=exerciseReasonsProbability)

df['ExperienceCategory'] = assignment
df['Age'] = ageDistribution
df['Age'] = df['Age'].clip(lower=18, upper=85)
df['Sport Preference'] = np.random.choice(sportNames, size = statSize, p=sportOdds)
df['Age Range'] = df['Age'].apply(assign_age_range)
df['ID'] = range(0, len(df))

print(df)



     ExperienceCategory   Age Sport Preference Age Range   ID
0    UPPER INTERMEDIATE  39.0             GOLF     36-45    0
1    LOWER INTERMEDIATE  18.0         SWIMMING     18-25    1
2    UPPER INTERMEDIATE  20.0       BASKETBALL     18-25    2
3    LOWER INTERMEDIATE  18.0          CRICKET     18-25    3
4    UPPER INTERMEDIATE  81.0         SWIMMING   Over 65    4
..                  ...   ...              ...       ...  ...
495        INTERMEDIATE  60.0       BASKETBALL     56-65  495
496  UPPER INTERMEDIATE  35.0         SWIMMING     26-35  496
497  LOWER INTERMEDIATE  43.0            RUGBY     36-45  497
498  UPPER INTERMEDIATE  18.0             GOLF     18-25  498
499        INTERMEDIATE  56.0       BASKETBALL     56-65  499

[500 rows x 5 columns]


In [94]:
#This is for developing a list of all the possible features

def prepend_strings(df, strings):
    new_df = df.copy()
    for i, col in enumerate(new_df.columns):
        new_df[col] = strings[i] + new_df[col].astype(str)
    return new_df

prepended_df = prepend_strings(df, ['skill:', 'age:', 'sport:', 'range:', 'id:'])

listOfExperienceFeatures = ['skill:ADVANCED', 'skill:LOWER INTERMEDIATE', 'skill:UPPER INTERMEDIATE', 'skill:INTERMEDIATE', 'skill:BEGINNER']
listOfAges=['age:' + (str(x) + ".0") for x in range(18, 86)] 
listOfSports=['sport:' + s.upper() for s in sportNames]
listOfAgeRanges=['range:'+ s for s in age_ranges]
listOfIds = ['id:' + str(x) for x in df['ID'].unique().tolist()]


user_feature_list = listOfExperienceFeatures + listOfSports + listOfAgeRanges + listOfAges + listOfIds
print(user_feature_list)

def df_to_tuples(df):
    result = []
    for index, row in df.iterrows():
        result.append((index, row.tolist()))
    return result

user_formatted = df_to_tuples(prepended_df)
print(user_formatted)


['skill:ADVANCED', 'skill:LOWER INTERMEDIATE', 'skill:UPPER INTERMEDIATE', 'skill:INTERMEDIATE', 'skill:BEGINNER', 'sport:CRICKET', 'sport:FOOTBALL', 'sport:VOLLEYBALL', 'sport:FIELD HOCKEY', 'sport:TENNIS', 'sport:BASKETBALL', 'sport:TABLE TENNIS', 'sport:BASEBALL', 'sport:TRACK AND FIELD', 'sport:GOLF', 'sport:RUGBY', 'sport:BADMINTON', 'sport:AMERICAN FOOTBALL', 'sport:SWIMMING', 'sport:GYMNASTICS', 'sport:CYCLING', 'sport:ICE HOCKEY', 'sport:HANDBALL', 'sport:ROCK CLIMBING', 'sport:FRISBEE', 'sport:BOWLING', 'range:18-25', 'range:26-35', 'range:36-45', 'range:46-55', 'range:56-65', 'range:Over 65', 'age:18.0', 'age:19.0', 'age:20.0', 'age:21.0', 'age:22.0', 'age:23.0', 'age:24.0', 'age:25.0', 'age:26.0', 'age:27.0', 'age:28.0', 'age:29.0', 'age:30.0', 'age:31.0', 'age:32.0', 'age:33.0', 'age:34.0', 'age:35.0', 'age:36.0', 'age:37.0', 'age:38.0', 'age:39.0', 'age:40.0', 'age:41.0', 'age:42.0', 'age:43.0', 'age:44.0', 'age:45.0', 'age:46.0', 'age:47.0', 'age:48.0', 'age:49.0', 'age:5

In [95]:
# Define the choices for the different fields
sports = [x.upper() for x in ['Cricket', 'Football', 'Volleyball', 'Field Hockey', 'Tennis', 'Basketball', 'Table Tennis', 'Baseball', 'Track and Field', 'Golf', 'Rugby', 'Badminton', 'American Football', 'Swimming', 'Gymnastics', 'Cycling', 'Ice Hockey', 'Handball', 'Rock Climbing', 'Frisbee', 'Bowling']]
skill_levels = ['ADVANCED', 'LOWER INTERMEDIATE', 'UPPER INTERMEDIATE', 'INTERMEDIATE', 'BEGINNER']
age_ranges = ['18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']

# Define the number of games to generate
num_games = statSize

# Generate random games
game_data = []
for i in range(num_games):
    # Generate a random time using a normal distribution
    game_time = np.random.normal(12, 4)
    game_time = int(max(0, min(game_time, 23)))
    #game_time = datetime.time(int(game_time), random.randint(0, 59), random.randint(0, 59))
    oneAge = np.random.normal(mu, sigma, 1)
    indoutchoice = np.random.choice(indoorOutdoorChoices, size = 1, p=indoorOutdoorProbability)

    # Generate a random date
    game_date = datetime.date.today() + datetime.timedelta(days=random.randint(1, 365))

    game = {
        #'session_title': f'Game {i}',
        'sport_of_choice': random.choice(sports),
        'confirmed_players': random.randint(1, 22),
        'skill_rating': random.choice(skill_levels),
        'age_range': random.choice(age_ranges),
        #'game_description': f'Description for game {i}',
        'time': game_time,
        #'distance': random.randint(1, 35),
        #'indoorOutdoor': indoutchoice[0]
        'ID': i
    }

    game_data.append(game)

games_df = pd.DataFrame(game_data)
print(games_df)

       sport_of_choice  confirmed_players        skill_rating age_range  time   
0            BADMINTON                 17  LOWER INTERMEDIATE     18-25     5  \
1             HANDBALL                 15            ADVANCED     46-55    11   
2              CRICKET                 12  LOWER INTERMEDIATE   Over 65    11   
3            BADMINTON                  1            ADVANCED     36-45     9   
4                RUGBY                 21        INTERMEDIATE     56-65     9   
..                 ...                ...                 ...       ...   ...   
495  AMERICAN FOOTBALL                 17            ADVANCED     46-55     9   
496       TABLE TENNIS                  6            ADVANCED   Over 65    17   
497          BADMINTON                 11  UPPER INTERMEDIATE     18-25    14   
498       TABLE TENNIS                 13            BEGINNER   Over 65     7   
499         ICE HOCKEY                 20        INTERMEDIATE     36-45     8   

      ID  
0      0  
1    

In [96]:
#distance_feature = ["dist:" + str(x) for x in range(1,36)]
playerCount_feature = ["count:" + str(x) for x in range (1,23)]
time_feature = ["time:"+str(x) for x in range(0,24)]
item_list_of_ids = ['id:' + str(x) for x in games_df['ID'].unique().tolist()]

item_feature_list = playerCount_feature + time_feature + listOfSports + listOfExperienceFeatures + listOfAgeRanges + item_list_of_ids

prepended_games = prepend_strings(games_df, ['sport:', 'count:', 'skill:', 'range:', 'time:',  'id:'])
formatted_games = df_to_tuples(prepended_games)

print(item_feature_list)
print(formatted_games)

['count:1', 'count:2', 'count:3', 'count:4', 'count:5', 'count:6', 'count:7', 'count:8', 'count:9', 'count:10', 'count:11', 'count:12', 'count:13', 'count:14', 'count:15', 'count:16', 'count:17', 'count:18', 'count:19', 'count:20', 'count:21', 'count:22', 'time:0', 'time:1', 'time:2', 'time:3', 'time:4', 'time:5', 'time:6', 'time:7', 'time:8', 'time:9', 'time:10', 'time:11', 'time:12', 'time:13', 'time:14', 'time:15', 'time:16', 'time:17', 'time:18', 'time:19', 'time:20', 'time:21', 'time:22', 'time:23', 'sport:CRICKET', 'sport:FOOTBALL', 'sport:VOLLEYBALL', 'sport:FIELD HOCKEY', 'sport:TENNIS', 'sport:BASKETBALL', 'sport:TABLE TENNIS', 'sport:BASEBALL', 'sport:TRACK AND FIELD', 'sport:GOLF', 'sport:RUGBY', 'sport:BADMINTON', 'sport:AMERICAN FOOTBALL', 'sport:SWIMMING', 'sport:GYMNASTICS', 'sport:CYCLING', 'sport:ICE HOCKEY', 'sport:HANDBALL', 'sport:ROCK CLIMBING', 'sport:FRISBEE', 'sport:BOWLING', 'skill:ADVANCED', 'skill:LOWER INTERMEDIATE', 'skill:UPPER INTERMEDIATE', 'skill:INTERM

In [97]:
ratings = np.zeros((len(df), len(games_df)))

base = 0.25
p = 0.07

def age_range_diff(range1, range2):
    mapping = {
        '18-25': 0,
        '26-35': 1,
        '36-45': 2,
        '46-55': 3,
        '56-65': 4,
        'Over 65': 5
    }
    diff = abs(mapping[range1] - mapping[range2])
    punishment = (2 / (1 + np.exp(-diff))) - 1
    return punishment

def skill_rating_diff(rating1, rating2):
    mapping = {
        'BEGINNER': 0,
        'LOWER INTERMEDIATE': 1,
        'INTERMEDIATE': 2,
        'UPPER INTERMEDIATE': 4,
        'ADVANCED': 4
    }
    diff = abs(mapping[rating1] - mapping[rating2])
    punishment = (2 / (1 + np.exp(-diff))) - 1
    return punishment

def distance_rating(distance):
    rating = 1 / (1 + np.exp(distance - 15))
    return rating

ratings = []
for i, user in df.iterrows():
    for j, game in games_df.iterrows():
        if (game['sport_of_choice'] == user['Sport Preference']) or (random.random() < p):
            rating = 0
            if(game['sport_of_choice'] == user['Sport Preference']):
                rating = 3
            #distance = game['distance']
            #rating += distance_rating(distance)

            if 5 <= game['confirmed_players'] <= 15:
                rating += 1
                
            age_range_difference = age_range_diff(game['age_range'], user['Age Range'])
            rating -= age_range_difference

            skill_rating_difference = skill_rating_diff(game['skill_rating'], user['ExperienceCategory'])
            rating -= skill_rating_difference

            if game['time'] < 12 or (game['time'] >= 13 and game['time'] < 18) and game['time'] >=6:
                rating += 1

            rating = min(max(rating, 0), 5)

            ratings.append((i, j, rating))
        
ratings_df = pd.DataFrame(ratings, columns=['user', 'game', 'rating'])

In [98]:
dataset = Dataset()
dataset.fit(users=df.index.values,
            items=games_df.index.values,
            user_features=user_feature_list,
            item_features=item_feature_list)

user_features = dataset.build_user_features(user_formatted)
item_features = dataset.build_item_features(formatted_games)
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

print(item_feature_map)
print(user_feature_map)

interactions, weights = dataset.build_interactions([(x[0], x[1], x[2]) for x in ratings_df.values])
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=36)
train_weights, test_weights = random_train_test_split(weights, test_percentage=0.2, random_state=36)

model = LightFM(
    learning_rate=0.1,
    loss='warp',
    )

model.fit(
    train_interactions,
    item_features=item_features,
    user_features=user_features,
    sample_weight = train_weights,
    epochs=7,
    verbose=True)

train_precision = precision_at_k(model, interactions, k=5, user_features = user_features, item_features = item_features).mean()
test_precision = precision_at_k(model, test_interactions, k=5, user_features = user_features, item_features = item_features, train_interactions=train_interactions).mean()

train_auc = auc_score(model, interactions, user_features = user_features, item_features = item_features).mean()
test_auc = auc_score(model, test_interactions, user_features = user_features, item_features = item_features, train_interactions=train_interactions).mean()

print('Precision: train %.2f, Precision test %.2f' % (train_precision, test_precision))
print('AUC: train %.2f , AUC: test %.2f' % (train_auc, test_auc))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121,

In [83]:
#Let's predict fot a new user rq

user_feature_list = ['skill:BEGINNER', 'age:19.0', 'sport:FOOTBALL', 'range:18-25']

def format_newuser_input(user_feature_map, user_feature_list):
  num_features = len(user_feature_list)
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass

  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

new_user_features = format_newuser_input(user_feature_map, user_feature_list)
predictions = model.predict(0, np.arange(statSize), user_features=new_user_features)

top_games = games_df.iloc[np.argsort(-predictions)][:5]
print(top_games)


       sport_of_choice  confirmed_players        skill_rating age_range  time   
277           FOOTBALL                  2  LOWER INTERMEDIATE     18-25    16  \
491           FOOTBALL                  9            BEGINNER   Over 65    12   
268           FOOTBALL                  5            BEGINNER     26-35     7   
370  AMERICAN FOOTBALL                  8            ADVANCED     26-35     9   
135           FOOTBALL                 19            ADVANCED     36-45    10   

      ID  
277  277  
491  491  
268  268  
370  370  
135  135  
